In [1]:
import numpy as np
import math

rng1 = np.random.default_rng(seed=0)

def objective_function(x):
    """Keane's Bump function.
    n = dimension of the domain
    x = vector in the domain"""
    cos4 = 0
    cos2 = 1
    x_i = 0
    n = len(x)
    for i in range(n):
        cos4 += math.cos(x[i])**4
        cos2 *= math.cos(x[i])**2
        x_i += (i+1)*(x[i]**2)
    if x_i == 0:
        answer = None
    else:
        answer = np.abs((cos4 - 2*cos2)/math.sqrt(x_i))
    ## we are turning the maximization problem into a minimization problem
    return -1 * answer

def update_D(D, R):
    D =  0.9 * D + 0.21 * R
    min_step_size = 1e-3
    max_step_size = 1
    D = np.where(D < min_step_size, min_step_size, D)
    D = np.where(D > max_step_size, max_step_size, D)
    return D

def decrement_temperature_huang(T, accepted_solutions_at_T):
    alpha = max(0.5, np.exp(- 0.7 * T / np.std(accepted_solutions_at_T)))
    return T*alpha

def proposal(x, D, rng): ## Parks 1991
    while True:
        u = rng.uniform(-1, 1, size=len(x))
        x_new = x + np.dot(D, u)
        if np.all((x_new > 0) & (x_new < 10)):
            if np.sum(x_new) < 15 * len(x) / 2 and np.prod(x_new) > 0.75:
                break
    return x_new, u

def archive_function(x, archive, unchanged_counter):
    ## 10 best solutions 
    archive_limit = 10
    f = objective_function(x)
    if len(archive) < archive_limit:
        archive.append((x, f))
    else:
        archive.append((x, f))  # Update all elements
        archive.sort(key=lambda item: item[1])  # Sort the archive by objective function value
        archive = archive[:archive_limit]  # Keep only the top elements
    if np.array_equal(archive[0][0],x): ## if current solution is the best solution in archive, reset counter
        unchanged_counter = 0
    else:
        unchanged_counter += 1
    return archive, unchanged_counter

def find_Tin_White(Lk, n, rng): ## White 1984
    Tin = []
    for i in range(25):
        while True:
            x = np.random.uniform(0, 10, size=n)
            if np.all((x > 0) & (x < 10)):
                if np.sum(x) < 15 * n / 2 and np.prod(x) > 0.75:
                    break
        D = np.identity(n)
        acceptanced_objectives = []
        for j in range(Lk):
            u = rng.uniform(-1, 1, size=n)
            x_new = x + np.dot(D, u)
            delta = objective_function(x_new) - objective_function(x)
            R = np.abs(np.diag(np.dot(D, u)))
            acceptanced_objectives.append(delta)
            x = x_new
            D = update_D(D, R)
        Tin.append(np.std(acceptanced_objectives))
    return np.average(Tin)

def find_Tin_kirkpatrick(Lk, n, rng): ## Kirkpatrick 1984
    Tin = []
    for i in range(50):
        while True:
            x = np.random.uniform(0, 10, size=n)
            if np.all((x > 0) & (x < 10)):
                if np.sum(x) < 15 * n / 2 and np.prod(x) > 0.75:
                    break
        D = np.identity(n)
        accepted_deltas = []
        for j in range(Lk):
            u = rng.uniform(-1, 1, size=n)
            x_new = x + np.dot(D, u)
            delta = objective_function(x_new) - objective_function(x)
            R = np.abs(np.diag(np.dot(D, u)))
            if delta > 0:
                accepted_deltas.append(delta)
            x = x_new
            D = update_D(D, R)
        Tin.append(-1 * np.sum(accepted_deltas) / math.log(0.8))
    return np.average(Tin)

def main(x0, Lk, rng):
    x_prev = x0
    # T = find_Tin_White(Lk, len(x0), rng)
    T = find_Tin_kirkpatrick(Lk, len(x0), rng)
    # print(f"Initial temperature: {T}")
    Din = np.identity(len(x0))
    D = Din
    accepted_solutions = []
    archive = []
    counter =  0
    max_iter = 15000
    alpha = 0.95
    unchanged_counter = 0
    unchanged_threshold = 5000
    while counter < max_iter and unchanged_counter < unchanged_threshold:
        acceptances_at_T = 0
        accepted_solutions_at_T = []
        for i in range(Lk):
            x_new, u = proposal(x_prev, Din, rng)
            counter += 1
            R = np.abs(np.diag(np.dot(D, u)))
            delta = objective_function(x_new) - objective_function(x_prev)
            if delta < 0:
                x_prev = x_new
                acceptances_at_T += 1
                accepted_solutions.append(x_new)
                accepted_solutions_at_T.append(x_new)
                D = update_D(D, R)
                archive, unchanged_counter = archive_function(x_new, archive, unchanged_counter)
            else:
                step_size = np.sqrt(np.sum(R**2))
                p = np.exp(- (0.7 * delta) / (T * step_size))
                if rng.uniform() < p:
                    x_prev = x_new
                    acceptances_at_T += 1
                    accepted_solutions.append(x_new)
                    accepted_solutions_at_T.append(x_new)
                    D = update_D(D, R)
                    archive, unchanged_counter = archive_function(x_new, archive, unchanged_counter)
                else:
                    unchanged_counter += 1
            if acceptances_at_T / Lk >= 0.6:
                break
        # T = decrement_temperature(T, alpha)
        T = decrement_temperature_huang(T, accepted_solutions_at_T)
        # print(f"New temperature: {T}, acceptances at previous T: {acceptances_at_T/ Lk}, counter: {counter}")
    return accepted_solutions, archive

In [2]:
def run_simulation(Lk):
    n = 8 ## 2D
    while True:
        x = np.random.uniform(0, 10, size=n)
        if np.all((x > 0) & (x < 10)):
            if np.sum(x) < 15 * n / 2 and np.prod(x) > 0.75:
                break
    accepted_solutions, archive = main(x, Lk, rng1)
    return archive[0][0], archive[0][1]

Lks = [75]

for Lk in Lks:
    objectives = []
    for i in range(25):
        x, f = run_simulation(Lk)
        objectives.append(f)

print(objectives)

c:\Users\sharm\anaconda3\envs\pracop2\lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\sharm\anaconda3\envs\pracop2\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\sharm\anaconda3\envs\pracop2\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[-0.6583603756040516, -0.5906240578525713, -0.38287945420129077, -0.6529793063292608, -0.3675319977457562, -0.6594525037488064, -0.6135007336281512, -0.6734349596830317, -0.6671428770985995, -0.62587816844567, -0.6300740526499182, -0.6207433975526315, -0.6389621273015622, -0.5435514074502195, -0.5305094928227372, -0.6011660692519759, -0.5793756652527514, -0.4767340770608183, -0.48195841483347523, -0.5195074889832676, -0.5434647173737943, -0.48070405924818055, -0.43331018675092753, -0.5788667656911265, -0.6183884851894035]


In [3]:
print(np.mean(objectives))
print(np.std(objectives))
print(np.min(objectives))
print(np.max(objectives))


-0.5667640336699991
0.08649977053436778
-0.6734349596830317
-0.3675319977457562


In [5]:
import time 
start = time.time()
x, f = run_simulation(75)
end = time.time()
print(end - start)

c:\Users\sharm\anaconda3\envs\pracop2\lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\sharm\anaconda3\envs\pracop2\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\sharm\anaconda3\envs\pracop2\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


2.633540391921997
